In [1]:
import numpy as np
import _ccpm

from sqlalchemy import create_engine, inspect, ForeignKey, Column, Integer, Float, String, and_, null

from sqlalchemy.orm import relationship, remote, backref
from sqlalchemy.orm.collections import attribute_mapped_collection
from sqlalchemy.orm.session import Session

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [2]:
class NetworkModel(Base):
    __tablename__ = 'models'

    id          = Column(Integer, primary_key = True, autoincrement=True)
    name        = Column(String, nullable=False)
    description = Column(String)
    
    #Work breakdown structure
    wbs        = relationship('WBSRecord', backref='model', cascade = 'all, delete-orphan')
    wbs_root   = relationship('WBSRecord', viewonly=True,
                              primaryjoin='and_(WBSRecord.model_id == NetworkModel.id, WBSRecord.parent_id == null())')
    
    # Network model elements
    activities = relationship('Activity' , backref='model', cascade = 'all, delete-orphan') # AoN AoA
    links      = relationship('Link'     , backref='model', cascade = 'all, delete-orphan') # AoN  -
    events     = relationship('Event'    , backref='model', cascade = 'all, delete-orphan') #  -  AoA
    
    # Visualization elements
    nodes      = relationship('Node'     , backref='model', cascade = 'all, delete-orphan')
    edges      = relationship('Edge'     , backref='model', cascade = 'all, delete-orphan')
    
    #----------------------------------------------------------------------------------------------
    def __init__(self, name, description='', session=None):
        self.name        = name
        self.description = description
        
        if session:
            assert isinstance(session, Session)

            session.add(self)
            session.flush() #We've got an id now

    #----------------------------------------------------------------------------------------------
    def describe(self):
        print(self.id, ':', self.name, ':', self.description)
        
    #----------------------------------------------------------------------------------------------
    def refresh(self):
        sesion = inspect(self).session
        session.flush()
        session.expire(self)
    
    #----------------------------------------------------------------------------------------------
    def copy(self, name=None):
        if not name:
            cp = NetworkModel(description=self.description, 
                              name='Copy of: ' + self.name,
                              session=inspect(self).session)
        else:
            cp = NetworkModel(description=self.description, 
                              name=name,
                              session=inspect(self).session)
        for w in self.wbs_root:
            w.copy(model=cp)
        
        cp.links  = [l.copy(model=cp) for l in self.links]
        cp.events = [e.copy(model=cp) for e in self.events]
        
        cp.nodes  = [n.copy(model=cp) for n in self.nodes]
        cp.edges  = [e.copy(model=cp) for e in self.edges]

        inspect(self).session.flush()
        return cp
    
    #----------------------------------------------------------------------------------------------
    @property
    def _iwbs(self):
        return dict([(w.path, i) for i,w in enumerate(self.wbs)])
    
    def wbs_record(self, path):
        return self.wbs[self._iwbs[path]]

    #----------------------------------------------------------------------------------------------
    def delete_aoa(self):
        if not self.events:
            return
        self.events = []
        self.nodes  = []
        self.edges  = []
        real_activities = []
        for a in self.activities:
            if a.wbs:
                a.src = None
                a.dst = None
                real_activities.append(a)
        #Don't need dummys!
        self.activities = real_activities
    
    #----------------------------------------------------------------------------------------------
    def generate_aoa(self):
        self.delete_aoa()

        session = inspect(self).session
        session.flush()

        act_id = session.query(Activity.id).filter(Activity.model_id == self.id).all()
        act_id = np.array(tuple(zip(*act_id))[0])
        
        lnk_ids = session.query(Link.src_id, Link.dst_id).filter(Link.model_id == self.id).all()
        lnk_ids = tuple(zip(*lnk_ids))

        lnk_src = np.array(lnk_ids[0])
        lnk_dst = np.array(lnk_ids[1])
        
        #print(act_id, lnk_src, lnk_dst)
        
        status, net_src, net_dst, lnk_src, lnk_dst = _ccpm.compute_aoa(act_id, lnk_src, lnk_dst)
        assert 0 == status
        
        print(lnk_src, lnk_dst)
        
        #TODO: Update links 
        #TODO: Think how to update them!
        #TODO: This works but id deletes links first!!!
        #self.links = []
        #for i,s in enumerate(lnk_src):
        #    l = Link(model=self, src_id=int(s), dst_id=int(lnk_dst[i]))

        #В net_src и net_dst у нас сначала идут реальные работы, а потом - фиктивные
        na = len(act_id)
        d  = np.max(act_id)
        for i in range(len(net_src)):
            if i < na:
                self.activities[i].src_id = int(net_src[i])
                self.activities[i].dst_id = int(net_dst[i])
            else:
                #Add a dummy
                d += 1
                dummy = Activity(model=self, id=int(d))
                dummy.src_id = int(net_src[i])
                dummy.dst_id = int(net_dst[i])
                
        for i in range(np.max(net_dst)):
            event = Event(self, int(i + 1))
            
        self.refresh()

    #----------------------------------------------------------------------------------------------
    def delete_activity(self, activity):
        if activity:
            assert isinstance(activity, Activity)
            assert activity.model_id == self.id

            self.activities.remove(activity)
            self.delete_aoa()
            self.refresh()
    
    #----------------------------------------------------------------------------------------------
    def delete_wbs_record(self, path):
        i = self._iwbs[path]
        
        if self.wbs[i].has_activities():
            self.delete_aoa()

        del self.wbs[i]
        self.refresh()

    #----------------------------------------------------------------------------------------------
    def delete_link(self, link):
        if link:
            assert isinstance(link, Link)
            assert link.model_id == self.id
            self.delete_aoa()
            self.links.remove(link)
            self.refresh()
            
    #----------------------------------------------------------------------------------------------
    #def __delete__(self, instance):
    #    print('Deleted model: ', self.id)
    #    super().__delete__(instance)
            

###################################################################################################
class WBSRecord(Base):
    __tablename__ = 'wbs'

    id        = Column(Integer, primary_key=True, autoincrement=True)
    model_id  = Column(Integer, ForeignKey('models.id'))
    parent_id = Column(Integer, ForeignKey('wbs.id'))
    
    path      = Column(String, nullable=False, index=True)
    name      = Column(String, nullable=False)
    
    children = relationship("WBSRecord",
                            remote_side=parent_id,
                            back_populates='parent',
                            cascade='all'
                           )
    
    parent  = relationship("WBSRecord",
                            remote_side=id,
                            back_populates='children',
                           )
    
    activity = relationship('Activity', 
                            backref='wbs', 
                            cascade = 'all', 
                            uselist=False)
    
    #----------------------------------------------------------------------------------------------
    def __init__(self, name, parent=None, model=None):
        if parent:
            assert isinstance(parent, WBSRecord)
            model = parent.model

        assert isinstance(model, NetworkModel)

        self.name   = name
        self.parent = parent
        self.model  = model
        self.path   = ''

        inspect(self).session.flush()
        # We've got an id now!

        if parent:
            self.path += parent.path + '.'
        self.path += str(self.id)

    #----------------------------------------------------------------------------------------------
    def _propagate_path(self, new_path_base):
        if self.parent:
            self.path = self.parent.path + '.' + str(self.id)
        else:
            self.path = str(self.id)

        for c in self.children:
            c._propagate_path(self.path)

    #----------------------------------------------------------------------------------------------
    def move_to(self, new_path_base):
        assert isinstance(new_path_base, str)
        
        if new_path_base != '':
            if new_path_base.startswith(self.path):
                raise ValueError('WTF R U doing motherfucker???')
            self.parent = self.model.comment(new_path_base)
        else:
            self.parent = None

        self._propagate_path(new_path_base)

        inspect(self).session.flush()
        
    #----------------------------------------------------------------------------------------------
    def swap(self, wbs):
        assert isinstance(wbs, WBSRecord)
        
        assert wbs.activity
        assert isinstance(wbs.activity, Activity)
        
        assert self.activity
        assert isinstance(self.activity, Activity)
        
        temp = self.name
        self.name = wbs.name
        wbs.name  = temp
        
        temp = self.activity
        self.activity = wbs.activity
        wbs.activity  = temp
        
        inspect(self).session.flush()
        
    #----------------------------------------------------------------------------------------------
    def copy(self, parent=None, model=None):
        cp = WBSRecord(self.name, parent=parent, model=model)

        if isinstance(self.activity, Activity):
            cp.activity = self.activity.copy(cp)

        cp.children = [c.copy(parent=cp) for c in self.children]

        return cp
    
    #----------------------------------------------------------------------------------------------
    def has_activities(self):
        if self.activity:
            assert isinstance(self.activity, Activity)
            return True
        
        for c in self.children:
            if c.has_activities():
                return True
        
        return False

    #----------------------------------------------------------------------------------------------
    def __repr__(self):
        return 'WBSRecord(model_id=%r id=%r, path=%r, name=%r)' % (
            self.model_id,
            self.id,
            self.path,
            self.name
        )

    #----------------------------------------------------------------------------------------------
    def dump(self, _level=0):
        return (
                '   ' * _level
                + repr(self)
                + "\n"
                + "".join([c.dump(_level + 1) for c in self.children])
        )

###################################################################################################
class Activity(Base):
    __tablename__ = 'activities'

    
    model_id = Column(Integer, ForeignKey('models.id'), primary_key=True)
    id       = Column(Integer,                          primary_key=True)
    
    wbs_id   = Column(Integer, ForeignKey('wbs.id'))
        
    in_links  = relationship('Link', 
                             primaryjoin='and_(Activity.model_id == Link.model_id, Activity.id == Link.src_id)',
                             backref='src', cascade='all, delete-orphan')
    out_links = relationship('Link', 
                             primaryjoin='and_(Activity.model_id == Link.model_id, Activity.id == Link.dst_id)', 
                             backref='dst', cascade='all, delete-orphan')

    #Работы являются связями для событий
    src_id    = Column(Integer, ForeignKey('events.id'))
    dst_id    = Column(Integer, ForeignKey('events.id'))
    
    #CPM Data
    duration    = Column(Float, default=0.0) # Длительность работы
    early_start = Column(Float, default=0.0) # Ранний старт
    late_start  = Column(Float, default=0.0) # Поздний старт
    early_end   = Column(Float, default=0.0) # Ранний финиш
    late_end    = Column(Float, default=0.0) # Поздний финиш
    reserve     = Column(Float, default=0.0) # Резерв времени
    
    def __init__(self, wbs=None, model=None, id=None, src=None, dst=None, src_id=None, dst_id=None):
        if wbs:
            assert isinstance(wbs, WBSRecord)
            model = wbs.model
        else:
            assert isinstance(id, int)
        
        assert isinstance(model, NetworkModel)

        if src:
            assert isinstance(src, Event)
            src_id = src.id

        if dst:
            assert isinstance(dst, Event)
            dst_id = dst.id

        self.model  = model
        #Обеспечили уникальность при генерации и возможность копирования
        self.id     = id if id else wbs.id
        self.wbs    = wbs
        self.src_id = src_id
        self.dst_id = dst_id

        model.delete_aoa()
        inspect(self).session.flush()
        
    #----------------------------------------------------------------------------------------------
    def __delete__(self, instance):
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print('Will delete:', self)
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        super().__delete__(instance)
        
    #----------------------------------------------------------------------------------------------
    def __repr__(self):
        if self.wbs:
            return 'Activity(model_id=%r id=%r, src_id=%r, dst_id=%r, wbs.path=%r, wbs.name=%r)' % (
                self.model_id,
                self.id,
                self.src_id,
                self.dst_id,
                self.wbs.path,
                self.wbs.name
            )
        else:
            return 'Activity(model_id=%r id=%r, src_id=%r, dst_id=%r, wbs.path=%r, wbs.name=%r)' % (
                self.model_id,
                self.id,
                self.src_id,
                self.dst_id,
                '',
                'Dummy'
            )
    
    #----------------------------------------------------------------------------------------------
    def copy(self, wbs):
        return Activity(wbs, src=self.src, dst=self.dst, id=self.id)

###################################################################################################
class Link(Base):
    __tablename__ = 'links'

    model_id  = Column(Integer, ForeignKey('models.id'    ),  primary_key=True)
    src_id    = Column(Integer, ForeignKey('activities.id'),  primary_key=True)
    dst_id    = Column(Integer, ForeignKey('activities.id'),  primary_key=True)
    
    def _clean_aoa(self):
        if self.model:
            assert isinstance(self.model, NetworkModel)
            self.model.delete_aoa()
            inspect(self).session.flush()
    
    def __init__(self, model=None, src=None, dst=None, src_id=None, dst_id=None):

        if model:
            assert isinstance(model, NetworkModel)
        
        if src:
            assert isinstance(src, Activity)
            src_id = src.id
            
        if dst:
            assert isinstance(dst, Activity)
            dst_id = dst.id
            
        if src and dst:
            assert src.model_id == dst.model_id

        if not model:
            model = src.model
                
        self.model    = model
        self.src_id   = src_id
        self.dst_id   = dst_id
        
        self._clean_aoa()

    #----------------------------------------------------------------------------------------------
    def set_src(self, node):
        assert isinstance(node, Node)
        assert node.model_id == self.model_id
        
        if node.id != self.src_id:
            self.src = node
            self._clean_aoa()
            inspect(self).session.flush()
            
    #----------------------------------------------------------------------------------------------
    def set_dst(self, node):
        assert isinstance(node, Node)
        assert node.model_id == self.model_id
        
        if node.id != self.dst_id:
            self.dst = node
            self._clean_aoa()
            inspect(self).session.flush()
        
    #----------------------------------------------------------------------------------------------
    #def __delete__(self, instance):
    #    self._clean_aoa()
    #    super().__delete__(instance)
            
    #----------------------------------------------------------------------------------------------
    def __repr__(self):
        return 'Link(model_id=%r src_id=%r, dst_id=%r)' % (
            self.model_id,
            self.src_id,
            self.dst_id
        )
    #----------------------------------------------------------------------------------------------
    def copy(self, model):
        return Link(src=self.src, dst=self.dst, model=model)

###################################################################################################
class Event(Base):
    __tablename__ = 'events'

    id       = Column(Integer,                          primary_key=True)
    model_id = Column(Integer, ForeignKey('models.id'), primary_key=True)

    in_activities  = relationship('Activity', 
                             primaryjoin='and_(Event.model_id == Activity.model_id, Event.id == Activity.src_id)',
                             backref='src')
    out_activities = relationship('Activity', 
                             primaryjoin='and_(Event.model_id == Activity.model_id, Event.id == Activity.dst_id)', 
                             backref='dst')
    
    early   = Column(Float, default=0.0) #Ранее время наступления
    late    = Column(Float, default=0.0) #Позднее время наступления
    reserve = Column(Float, default=0.0) #Резерв времени
    
    def __init__(self, model, id):
        assert isinstance(model, NetworkModel)
        
        self.model = model
        self.id = id

    #----------------------------------------------------------------------------------------------
    def __repr__(self):
        return 'Event(model_id=%r id=%r)' % (
            self.model_id,
            self.id
        )

    #----------------------------------------------------------------------------------------------
    def copy(self, model):
        return Event(model=model,id=self.id)
    
    def __delete__(self, instance):
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print('Will delete:', self)
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        super().__delete__(instance)
    
###################################################################################################
class Node(Base):
    __tablename__ = 'nodes'
    
    model_id = Column(Integer, ForeignKey('models.id'), primary_key=True)
    id       = Column(Integer,                          primary_key=True)
    
    event_id = Column(Integer, ForeignKey('events.id'))
    
    x = Column(Float, default=0.0)
    y = Column(Float, default=0.0)
        
    in_edges  = relationship('Edge', 
                             primaryjoin='and_(Node.model_id == Edge.model_id, Node.id == Edge.src_id)',
                             backref='src', cascade = 'all, delete-orphan')
    out_edges = relationship('Edge', 
                             primaryjoin='and_(Node.model_id == Edge.model_id, Node.id == Edge.dst_id)', 
                             backref='dst', cascade = 'all, delete-orphan')
    
    event = relationship('Event',
                         primaryjoin='and_(Node.model_id == Event.model_id, Node.event_id == Event.id)')
    
    def __init__(self, id, model=None,  event=None):
        if model:
            assert isinstance(model, NetworkModel)
        
        if event:
            assert isinstance(event, Event)
            self.event_id = event.id

            if not model:
                model = event.model

        self.model = model
        self.id    = id

    #----------------------------------------------------------------------------------------------
    def __repr__(self):
        return 'Node(model_id=%r, id=%r, event_id=%r)' % (
            self.model_id,
            self.id,
            self.event_id
        )
    
    #----------------------------------------------------------------------------------------------
    def copy(self, model):
        return Node(self.id, model=model, event=self.event)

###################################################################################################
class Edge(Base):
    __tablename__ = 'edges'

    model_id  = Column(Integer, ForeignKey('models.id'),  primary_key=True)
    src_id    = Column(Integer, ForeignKey('nodes.id') ,  primary_key=True)
    dst_id    = Column(Integer, ForeignKey('nodes.id') ,  primary_key=True)
    
    def __init__(self, model=None, src=None, dst=None, src_id=None, dst_id=None):

        if model:
            assert isinstance(model, NetworkModel)
        
        if src:
            assert isinstance(src, Node)
            src_id = src.id
            
        if dst:
            assert isinstance(dst, Node)
            dst_id = dst.id
            
        if src and dst:
            assert src.model_id == dst.model_id

        if not model:
            model = src.model
                
        self.model  = model
        self.src_id = src_id
        self.dst_id = dst_id
            
    #----------------------------------------------------------------------------------------------
    def __repr__(self):
        return 'Edge(model_id=%r src_id=%r, dst_id=%r)' % (
            self.model_id,
            self.src_id,
            self.dst_id
        )
    #----------------------------------------------------------------------------------------------
    def copy(self, model):
        return Edge(src=self.src, dst=self.dst, model=model)

In [3]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo = True)

In [4]:
from sqlalchemy.orm import sessionmaker
_Session = sessionmaker(bind=engine)
session = _Session()

In [5]:
Base.metadata.create_all(engine)

2021-09-05 20:52:43,057 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-05 20:52:43,058 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("models")
2021-09-05 20:52:43,058 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-05 20:52:43,059 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("models")
2021-09-05 20:52:43,060 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-05 20:52:43,061 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("wbs")
2021-09-05 20:52:43,062 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-05 20:52:43,062 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("wbs")
2021-09-05 20:52:43,063 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-05 20:52:43,065 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("activities")
2021-09-05 20:52:43,065 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-05 20:52:43,066 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("activities")
2021-09-05 20:52:43,067 INFO sqlalchemy.engine.Engine [raw sql] ()
202

In [6]:
nm = NetworkModel(name='First model!', session=session)

2021-09-05 20:52:43,259 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-05 20:52:43,260 INFO sqlalchemy.engine.Engine INSERT INTO models (name, description) VALUES (?, ?)
2021-09-05 20:52:43,261 INFO sqlalchemy.engine.Engine [generated in 0.00083s] ('First model!', '')


In [7]:
w1  = WBSRecord('Heating and frames study',                                 model=nm)
w2  = WBSRecord('Scouring and installation of building site establishment', model=nm)
w3  = WBSRecord('Earthwork and concrete well',                              model=nm)
w4  = WBSRecord('Earthwork and concrete longitudinal beams',                model=nm)
w5  = WBSRecord('Frame construction',                                       model=nm)
w6  = WBSRecord('Frame transport',                                          model=nm)
w7  = WBSRecord('Assemblage',                                               model=nm)
w8  = WBSRecord('Earthwork and pose drains',                                model=nm)
w9  = WBSRecord('Heating provisioning and assembly',                        model=nm)
w10 = WBSRecord('Electric installation',                                    model=nm)
w11 = WBSRecord('Painting',                                                 model=nm)
w12 = WBSRecord('Pavement',                                                 model=nm)

2021-09-05 20:52:43,342 INFO sqlalchemy.engine.Engine INSERT INTO wbs (model_id, parent_id, path, name) VALUES (?, ?, ?, ?)
2021-09-05 20:52:43,343 INFO sqlalchemy.engine.Engine [generated in 0.00076s] (1, None, '', 'Heating and frames study')
2021-09-05 20:52:43,346 INFO sqlalchemy.engine.Engine UPDATE wbs SET path=? WHERE wbs.id = ?
2021-09-05 20:52:43,347 INFO sqlalchemy.engine.Engine [generated in 0.00059s] ('1', 1)
2021-09-05 20:52:43,348 INFO sqlalchemy.engine.Engine INSERT INTO wbs (model_id, parent_id, path, name) VALUES (?, ?, ?, ?)
2021-09-05 20:52:43,348 INFO sqlalchemy.engine.Engine [cached since 0.005962s ago] (1, None, '', 'Scouring and installation of building site establishment')
2021-09-05 20:52:43,351 INFO sqlalchemy.engine.Engine UPDATE wbs SET path=? WHERE wbs.id = ?
2021-09-05 20:52:43,352 INFO sqlalchemy.engine.Engine [cached since 0.00539s ago] ('2', 2)
2021-09-05 20:52:43,352 INFO sqlalchemy.engine.Engine INSERT INTO wbs (model_id, parent_id, path, name) VALUES 

In [8]:
#TODO: Добавить проверку на наличие работы у элемента ИСР
a1  = Activity(wbs=w1)
a2  = Activity(wbs=w2)
a3  = Activity(wbs=w3)
a4  = Activity(wbs=w4)
a5  = Activity(wbs=w5)
a6  = Activity(wbs=w6)
a7  = Activity(wbs=w7)
a8  = Activity(wbs=w8)
a9  = Activity(wbs=w9)
a10 = Activity(wbs=w10)
a11 = Activity(wbs=w11)
a12 = Activity(wbs=w12)

2021-09-05 20:52:43,447 INFO sqlalchemy.engine.Engine SELECT activities.model_id AS activities_model_id, activities.id AS activities_id, activities.wbs_id AS activities_wbs_id, activities.src_id AS activities_src_id, activities.dst_id AS activities_dst_id, activities.duration AS activities_duration, activities.early_start AS activities_early_start, activities.late_start AS activities_late_start, activities.early_end AS activities_early_end, activities.late_end AS activities_late_end, activities.reserve AS activities_reserve 
FROM activities 
WHERE ? = activities.wbs_id
2021-09-05 20:52:43,447 INFO sqlalchemy.engine.Engine [generated in 0.00072s] (1,)
2021-09-05 20:52:43,452 INFO sqlalchemy.engine.Engine UPDATE wbs SET path=? WHERE wbs.id = ?
2021-09-05 20:52:43,452 INFO sqlalchemy.engine.Engine [cached since 0.1062s ago] ('12', 12)
2021-09-05 20:52:43,454 INFO sqlalchemy.engine.Engine INSERT INTO activities (model_id, id, wbs_id, src_id, dst_id, duration, early_start, late_start, early

In [9]:
#Связи
l1_5 = Link(src=a1, dst=a5)
l2_5 = Link(src=a2, dst=a5)
l3_5 = Link(src=a3, dst=a5)

l2_6 = Link(src=a2, dst=a6)
l3_6 = Link(src=a3, dst=a6)

l3_7 = Link(src=a3, dst=a7)
l4_7 = Link(src=a4, dst=a7)

l1_8 = Link(src=a1, dst=a8)
l6_8 = Link(src=a6, dst=a8)
l7_8 = Link(src=a7, dst=a8)

l5_9 = Link(src=a5, dst=a9)
l6_9 = Link(src=a6, dst=a9)
l7_9 = Link(src=a7, dst=a9)

l3_10 = Link(src=a3, dst=a10)
l6_10 = Link(src=a6, dst=a10)
l7_10 = Link(src=a7, dst=a10)

l6_11 = Link(src=a6, dst=a11)
l8_11 = Link(src=a8, dst=a11)
l9_11 = Link(src=a9, dst=a11)

l7_12  = Link(src=a7,  dst=a12)
l8_12  = Link(src=a8,  dst=a12)
l9_12  = Link(src=a9,  dst=a12)
l10_12 = Link(src=a10, dst=a12)

2021-09-05 20:52:43,543 INFO sqlalchemy.engine.Engine INSERT INTO links (model_id, src_id, dst_id) VALUES (?, ?, ?)
2021-09-05 20:52:43,544 INFO sqlalchemy.engine.Engine [generated in 0.00093s] (1, 1, 5)
2021-09-05 20:52:43,546 INFO sqlalchemy.engine.Engine INSERT INTO links (model_id, src_id, dst_id) VALUES (?, ?, ?)
2021-09-05 20:52:43,547 INFO sqlalchemy.engine.Engine [cached since 0.00412s ago] (1, 2, 5)
2021-09-05 20:52:43,551 INFO sqlalchemy.engine.Engine INSERT INTO links (model_id, src_id, dst_id) VALUES (?, ?, ?)
2021-09-05 20:52:43,552 INFO sqlalchemy.engine.Engine [cached since 0.009006s ago] (1, 3, 5)
2021-09-05 20:52:43,553 INFO sqlalchemy.engine.Engine INSERT INTO links (model_id, src_id, dst_id) VALUES (?, ?, ?)
2021-09-05 20:52:43,554 INFO sqlalchemy.engine.Engine [cached since 0.0114s ago] (1, 2, 6)
2021-09-05 20:52:43,556 INFO sqlalchemy.engine.Engine INSERT INTO links (model_id, src_id, dst_id) VALUES (?, ?, ?)
2021-09-05 20:52:43,556 INFO sqlalchemy.engine.Engine [c

In [10]:
nm.generate_aoa()
nm.generate_aoa()#Test 2 times

2021-09-05 20:52:43,668 INFO sqlalchemy.engine.Engine SELECT activities.id AS activities_id 
FROM activities 
WHERE activities.model_id = ?
2021-09-05 20:52:43,668 INFO sqlalchemy.engine.Engine [generated in 0.00087s] (1,)
2021-09-05 20:52:43,671 INFO sqlalchemy.engine.Engine SELECT links.src_id AS links_src_id, links.dst_id AS links_dst_id 
FROM links 
WHERE links.model_id = ?
2021-09-05 20:52:43,671 INFO sqlalchemy.engine.Engine [generated in 0.00085s] (1,)
[ 1  2  3  4  5  6  7  8  9 10 11 12] [ 1  1  2  2  3  3  3  3  4  5  6  6  6  6  7  7  7  7  8  8  9  9 10] [ 5  8  5  6  5  6  7 10  7  9  8  9 10 11  8  9 10 12 11 12 11 12 12]
[ 1  2  3  2  3  3  4  1  6  7  5  6  7  6  7  8  9  8  9 10] [ 5  5  5  6  6  7  7  8  8  8  9  9  9 10 10 11 11 12 12 12]
2021-09-05 20:52:43,675 INFO sqlalchemy.engine.Engine SELECT activities.model_id AS activities_model_id, activities.id AS activities_id, activities.wbs_id AS activities_wbs_id, activities.src_id AS activities_src_id, activities.dst_

In [11]:
for w in nm.wbs:
    print(w)

2021-09-05 20:52:43,924 INFO sqlalchemy.engine.Engine SELECT models.id AS models_id, models.name AS models_name, models.description AS models_description 
FROM models 
WHERE models.id = ?
2021-09-05 20:52:43,925 INFO sqlalchemy.engine.Engine [cached since 0.1968s ago] (1,)
2021-09-05 20:52:43,927 INFO sqlalchemy.engine.Engine SELECT wbs.id AS wbs_id, wbs.model_id AS wbs_model_id, wbs.parent_id AS wbs_parent_id, wbs.path AS wbs_path, wbs.name AS wbs_name 
FROM wbs 
WHERE ? = wbs.model_id
2021-09-05 20:52:43,928 INFO sqlalchemy.engine.Engine [generated in 0.00075s] (1,)
WBSRecord(model_id=1 id=1, path='1', name='Heating and frames study')
WBSRecord(model_id=1 id=2, path='2', name='Scouring and installation of building site establishment')
WBSRecord(model_id=1 id=3, path='3', name='Earthwork and concrete well')
WBSRecord(model_id=1 id=4, path='4', name='Earthwork and concrete longitudinal beams')
WBSRecord(model_id=1 id=5, path='5', name='Frame construction')
WBSRecord(model_id=1 id=6, pa

In [12]:
for a in nm.activities:
    print(a)

2021-09-05 20:52:44,048 INFO sqlalchemy.engine.Engine SELECT activities.model_id AS activities_model_id, activities.id AS activities_id, activities.wbs_id AS activities_wbs_id, activities.src_id AS activities_src_id, activities.dst_id AS activities_dst_id, activities.duration AS activities_duration, activities.early_start AS activities_early_start, activities.late_start AS activities_late_start, activities.early_end AS activities_early_end, activities.late_end AS activities_late_end, activities.reserve AS activities_reserve 
FROM activities 
WHERE ? = activities.model_id
2021-09-05 20:52:44,049 INFO sqlalchemy.engine.Engine [cached since 0.3739s ago] (1,)
Activity(model_id=1 id=1, src_id=1, dst_id=4, wbs.path='1', wbs.name='Heating and frames study')
Activity(model_id=1 id=2, src_id=1, dst_id=3, wbs.path='2', wbs.name='Scouring and installation of building site establishment')
Activity(model_id=1 id=3, src_id=1, dst_id=2, wbs.path='3', wbs.name='Earthwork and concrete well')
Activity(m

In [13]:
for e in nm.events:
    print(e)

2021-09-05 20:52:44,137 INFO sqlalchemy.engine.Engine SELECT events.id AS events_id, events.model_id AS events_model_id, events.early AS events_early, events.late AS events_late, events.reserve AS events_reserve 
FROM events 
WHERE ? = events.model_id
2021-09-05 20:52:44,138 INFO sqlalchemy.engine.Engine [cached since 0.6801s ago] (1,)
Event(model_id=1 id=1)
Event(model_id=1 id=2)
Event(model_id=1 id=3)
Event(model_id=1 id=4)
Event(model_id=1 id=5)
Event(model_id=1 id=6)
Event(model_id=1 id=7)
Event(model_id=1 id=8)
Event(model_id=1 id=9)
Event(model_id=1 id=10)
Event(model_id=1 id=11)


In [14]:
for l in session.query(Link).order_by(Link.dst_id).all():
    print(l)

2021-09-05 20:52:44,226 INFO sqlalchemy.engine.Engine SELECT links.model_id AS links_model_id, links.src_id AS links_src_id, links.dst_id AS links_dst_id 
FROM links ORDER BY links.dst_id
2021-09-05 20:52:44,226 INFO sqlalchemy.engine.Engine [generated in 0.00079s] ()
Link(model_id=1 src_id=1, dst_id=5)
Link(model_id=1 src_id=2, dst_id=5)
Link(model_id=1 src_id=3, dst_id=5)
Link(model_id=1 src_id=2, dst_id=6)
Link(model_id=1 src_id=3, dst_id=6)
Link(model_id=1 src_id=3, dst_id=7)
Link(model_id=1 src_id=4, dst_id=7)
Link(model_id=1 src_id=1, dst_id=8)
Link(model_id=1 src_id=6, dst_id=8)
Link(model_id=1 src_id=7, dst_id=8)
Link(model_id=1 src_id=5, dst_id=9)
Link(model_id=1 src_id=6, dst_id=9)
Link(model_id=1 src_id=7, dst_id=9)
Link(model_id=1 src_id=3, dst_id=10)
Link(model_id=1 src_id=6, dst_id=10)
Link(model_id=1 src_id=7, dst_id=10)
Link(model_id=1 src_id=6, dst_id=11)
Link(model_id=1 src_id=8, dst_id=11)
Link(model_id=1 src_id=9, dst_id=11)
Link(model_id=1 src_id=7, dst_id=12)
Link